# Conexão com banco

In [313]:
import psycopg2
import pandas as pd
import psycopg2
from IPython.display import display

host = "silly.db.elephantsql.com"
port = "5432"
database = "ljppladj"
user = "ljppladj"
password = "JFgtpa3EjnB6zOJ75ejGn_gUOl8-dy3j"

try:
    conn = psycopg2.connect(
        host=host,
        port=port,
        database=database,
        user=user,
        password=password
    )
    print("Conexão bem-sucedida!")
except Exception as e:
    print(f"Erro ao conectar: {e}")


Conexão bem-sucedida!


# Populando dataframe

In [319]:
# Crie um cursor para executar comandos SQL no banco de dados
cursor = conn.cursor()

# Execute a consulta SQL para juntar as tabelas e obter os dados desejados
cursor.execute("""
  SELECT 
        candidates.cnd_id, 
        candidates.cnd_name, 
        candidates.cnd_email, 
        candidates.cnd_current_profession, 
        candidates.cnd_experience, 
        skl_skill.skl_name as skill,
        knw_knowledge.knw_name as knowledge,
        att_attitude.att_name as attitude
    FROM candidates
    LEFT JOIN candidate_skill ON candidates.cnd_id = candidate_skill.cnd_id
    LEFT JOIN skl_skill ON candidate_skill.knw_id = skl_skill.skl_id
    LEFT JOIN knw_knowledge ON skl_skill.skl_id = knw_knowledge.knw_id
    LEFT JOIN candidate_attitude ON candidates.cnd_id = candidate_attitude.cnd_id
    LEFT JOIN att_attitude ON candidate_attitude.knw_id = att_attitude.att_id

""")

df = pd.DataFrame(cursor.fetchall(), columns=[desc[0] for desc in cursor.description])

# Remover duplicatas mantendo apenas as colunas relevantes
df_skills = df[['cnd_id', 'cnd_name', 'cnd_experience', 'skill']].drop_duplicates()
df_knowledge = df[['cnd_id', 'cnd_name', 'cnd_experience', 'knowledge']].drop_duplicates()
df_attitude = df[['cnd_id', 'cnd_name', 'cnd_experience', 'attitude']].drop_duplicates()

display(df_skills)
# display(df_knowledge)
# display(df_attitude)

cursor.close()

,cnd_id,cnd_name,cnd_email,cnd_current_profession,cnd_experience,skill,knowledge,attitude
0,1,João,joao@email.com,Estagiário,0,Desenvolvimento de Interfaces com HTML/CSS,None,Atenção ao Cliente Interno
1,1,João,joao@email.com,Estagiário,0,Desenvolvimento de Interfaces com HTML/CSS,None,Criatividade
2,1,João,joao@email.com,Estagiário,0,Desenvolvimento de Interfaces com HTML/CSS,None,Pensamento Analítico
3,1,João,joao@email.com,Estagiário,0,Desenvolvimento de Interfaces com HTML/CSS,None,Ética Profissional
4,1,João,joao@email.com,Estagiário,0,Desenvolvimento de Interfaces com HTML/CSS,None,Comunicação Clara
...,...,...,...,...,...,...,...,...
972,21,Ricardo Alves Silva,ricardo.alves.silva@email.com,Desenvolvedor Sênior,2,Machine Learning,None,Orientação ao Cliente
973,21,Ricardo Alves Silva,ricardo.alves.silva@email.com,Desenvolvedor Sênior,2,Machine Learning,None,Empatia
974,21,Ricardo Alves Silva,ricardo.alves.silva@email.com,Desenvolvedor Sênior,2,Machine Learning,None,Foco em Resultados
975,21,Ricardo Alves Silva,ricardo.alves.silva@email.com,Desenvolvedor Sênior,2,Machine Learning,None,Adaptação


# Tratamento de dados

In [315]:
desenvolvedor = {
    'profissao': 'Desenvolvedor Back-End',
    'habilidades': [
        'Python', 
        'Java', 
        'Node.js', 
        'MySQL',
        'PostgreSQL'
    ],
    'conhecimentos': [
        'Manipulação de Dados',
        'Conhecimento em SQL',
    ],
    'atitudes': [
        'Lógica de Programação',
        'Resolução de Problemas',
        'Criatividade',
    ]
}


habilidades = desenvolvedor['habilidades']
conhecimentos = desenvolvedor['conhecimentos']
atitudes = desenvolvedor['atitudes']

df_skills['tem_habilidade'] = 0
df_knowledge['tem_conhecimento'] = 0
df_attitude['tem_atitude'] = 0

# Iterar sobre as habilidades e marcar como 1 se presente na coluna 'skill'
# Para as habilidades
for habilidade in habilidades:
    df_skills['tem_habilidade'] = (df_skills['tem_habilidade'] | df_skills['skill'].fillna('').str.contains(habilidade, case=False)).astype(int)

# Para os conhecimentos
for conhecimento in conhecimentos:
    df_knowledge['tem_conhecimento'] = (df_knowledge['tem_conhecimento'] | df_knowledge['knowledge'].fillna('').str.contains(conhecimento, case=False)).astype(int)

# Para as atitudes
for atitude in atitudes:
    df_attitude['tem_atitude'] = (df_attitude['tem_atitude'] | df_attitude['attitude'].fillna('').str.contains(atitude, case=False)).astype(int)


# display(df_skills)
# display(df_knowledge)
# display(df_attitude)


# Treinamento dos dados

In [316]:
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler

# Definindo X com as características relevantes para habilidades
X_habilidades = df_skills[['cnd_id', 'cnd_experience', 'tem_habilidade']]
X_habilidades['num_habilidades'] = X_habilidades['tem_habilidade'].sum()
df_skills['num_habilidades'] = df_skills.groupby('cnd_id')['tem_habilidade'].transform('sum')

# Definindo X com as características relevantes para atitudes
X_atitudes = df_attitude[['cnd_id', 'cnd_experience', 'tem_atitude']]
X_atitudes['num_atitudes'] = X_atitudes['tem_atitude'].sum()
df_attitude['num_atitudes'] = df_attitude.groupby('cnd_id')['tem_atitude'].transform('sum')

# Definindo X com as características relevantes para conhecimentos
X_conhecimentos = df_knowledge[['cnd_id', 'cnd_experience', 'tem_conhecimento']]
X_conhecimentos['num_conhecimentos'] = X_conhecimentos['tem_conhecimento'].sum()
df_knowledge['num_conhecimentos'] = df_knowledge.groupby('cnd_id')['tem_conhecimento'].transform('sum')

# Normalizando os dados para que todas as características tenham a mesma escala
scaler = StandardScaler()
X_habilidades_normalized = scaler.fit_transform(X_habilidades)
X_atitudes_normalized = scaler.fit_transform(X_atitudes)
X_conhecimentos_normalized = scaler.fit_transform(X_conhecimentos)

# Aplicando K-Means para clusterizar os candidatos (habilidades)
kmeans_habilidades = KMeans(n_clusters=2, random_state=42)
clusters_habilidades = kmeans_habilidades.fit_predict(X_habilidades_normalized)
df_skills['Cluster_Habilidades'] = clusters_habilidades

# Aplicando K-Means para clusterizar os candidatos (atitudes)
kmeans_atitudes = KMeans(n_clusters=2, random_state=42)
clusters_atitudes = kmeans_atitudes.fit_predict(X_atitudes_normalized)
df_attitude['Cluster_Atitudes'] = clusters_atitudes

# Aplicando K-Means para clusterizar os candidatos (conhecimentos)
kmeans_conhecimentos = KMeans(n_clusters=2, random_state=42)
clusters_conhecimentos = kmeans_conhecimentos.fit_predict(X_conhecimentos_normalized)
df_knowledge['Cluster_Conhecimentos'] = clusters_conhecimentos

# # Exibindo os candidatos de cada cluster
# for cluster in df_skills['Cluster_Habilidades'].unique():
#     print(f'Candidatos no Cluster de Habilidades {cluster}:')
#     print(df_skills[df_skills['Cluster_Habilidades'] == cluster])

# for cluster in df_attitude['Cluster_Atitudes'].unique():
#     print(f'Candidatos no Cluster de Atitudes {cluster}:')
#     print(df_attitude[df_attitude['Cluster_Atitudes'] == cluster])

# for cluster in df_knowledge['Cluster_Conhecimentos'].unique():
#     print(f'Candidatos no Cluster de Conhecimentos {cluster}:')
#     print(df_knowledge[df_knowledge['Cluster_Conhecimentos'] == cluster])

# Exibindo os DataFrames finais
# display(df_skills)
# display(df_attitude)
# display(df_knowledge)


C:\Users\Tria-0155\AppData\Local\Temp\ipykernel_16676\2061638256.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_habilidades['num_habilidades'] = X_habilidades['tem_habilidade'].sum()
C:\Users\Tria-0155\AppData\Local\Temp\ipykernel_16676\2061638256.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_atitudes['num_atitudes'] = X_atitudes['tem_atitude'].sum()
C:\Users\Tria-0155\AppData\Local\Temp\ipykernel_16676\2061638256.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a sli

# Definindo Ranking

In [317]:
# Somando as contagens de habilidades, conhecimentos e atitudes
df_skills['num_habilidades'] = df_skills.groupby('cnd_id')['tem_habilidade'].transform('sum')
df_attitude['num_atitudes'] = df_attitude.groupby('cnd_id')['tem_atitude'].transform('sum')
df_knowledge['num_conhecimentos'] = df_knowledge.groupby('cnd_id')['tem_conhecimento'].transform('sum')

df_final = df_skills.copy()

# Somando as contagens totais
# Remover linhas com valores nulos em 'pontos_cha'
df_final['pontos_cha'] = df_skills['num_habilidades'] + df_attitude['num_atitudes'] + df_knowledge['num_conhecimentos']

df_final['num_atitudes'] = df_attitude['num_atitudes']
df_final['num_conhecimentos'] = df_knowledge['num_conhecimentos']
df_final = df_final.dropna(subset=['pontos_cha'])
# Remover as colunas 'skill' e 'cnd_experience'
df_final = df_final.drop(['skill', 'cnd_experience'], axis=1)

# display(df_final)
display(df_final.head(16))



ValueError: cannot reindex on an axis with duplicate labels